In [ ]:
import matplotlib.pylab as plt
import numpy as np
import scipy.integrate as integrate

# This will reload modules that have been edited
%load_ext autoreload
%autoreload 2


In [ ]:
# Uncomment when running on Colab
#!pip install git+https://github.com/mattbellis/earthshine

In [ ]:
import earthshine as etools 

In [ ]:
density = 1.6e30 # Number of nucleons per m3?
#density_m3 = density_kg * ( 6e26) # Number of nucleons per kg

length = 1 # length of "target" in meters?

CMS_length = 21 # meters
CMS_width = 15 # meters
area_of_CMS = CMS_length*CMS_width 

area_of_CMS /= 2 
# This is just because the cross section "looks" different from
# different angles, so this is just a guesstimate. 


plt.figure(figsize=(10,8))

markerstyles = ['o', 's', 'v', '^', 'P', 'D']

# Loop over different energies

for icount,elo in enumerate(range(20, 120, 20)):

    #elo = 100
    #ehi = 101
    ehi = elo + 1
    print(elo,ehi)
    
    energy = np.linspace(elo,ehi,1)
    print(energy)
    final_energies = np.linspace(1,200,10000)

    # Let's do the calculations for many chunks of rock
    N_at_CMS = np.zeros(len(final_energies))

    # These will be at different distances from CMS

    #plt.figure()
    for distance in range(1,200,1):

        # Calculate the number of neutrinos coming out for 1 m^2 of rock of length=length
        # per steradian, at some given energy
        N = np.ones(len(energy))

        # What is the solid angle at this distance?
        Omega = etools.solid_angle(CMS_length, CMS_width/2, distance)
        N *= Omega

        # How "many" of these 1m^2 chunks are there at distance=distance
        # *below* CMS (hemisphere)
        hemisphere_area = 2*np.pi*(distance**2) # in m^2
        N *= hemisphere_area

        #print(f"distance: {distance}   Omega: {Omega}     A_hemi: {hemisphere_area}")

        # Attenuation -- energy loss!
        N_at_CMS_from_this_distance = np.zeros(len(final_energies))
        for i,e in enumerate(energy):
            ke_final= etools.energy_loss_per_distance_traveled(e*1e9, distance, step_size=.1)
            ke_final /= 1e9
            #print(ke_final)

            # Fill the array that represents the final spectra
            #idx, central = etools.find_the_number(ke_final, energy)
            idx, central = etools.find_the_number(ke_final, final_energies)

            #print(e, ke_final, idx, central)
            if idx is not None:
              # N is the number coming out of the 1m^2 rock at distance=distance
              # N_at_CMS is the number of muons measured at CMS as different energies
              N_at_CMS_from_this_distance[idx] += N[i]
            #else:
              #print(e, ke_final, idx, central)

        #print(N_at_CMS_from_this_distance)
        N_at_CMS += N_at_CMS_from_this_distance

        # For debugging
        #if distance%20==0:
        #    plt.plot(final_energies,N_at_CMS_from_this_distance,label=f'distance={distance} m')
        #if distance%10==0:
        #   plt.plot(final_energies,N_at_CMS_from_this_distance,label=f'distance={distance} m')
    #plt.xscale('log')
    #plt.xlabel("energy",fontsize=14)
    #plt.ylabel("number",fontsize=14)
    ##plt.title("energy vs number at CMS (per second)")
    ##plt.savefig("energy_vs_number_per_second")
    ###plt.yscale('log')
    #plt.legend()



    # Now do for CMS 
    #plt.figure()
    #plt.plot(energy, N)
    
    plt.plot(final_energies, N_at_CMS,markerstyles[icount],label=fr'DM candidate mass {elo} GeV/c$^2$')
#plt.xscale('log')
#plt.yscale('log')
plt.xlabel('Energy of muons arriving at CMS (GeV)',fontsize=18)
plt.ylabel('Number of muons arriving at CMS (arb)',fontsize=18)

plt.legend(fontsize=14, markerscale=2)
plt.savefig('muon_spectra_earthshine.png')


In [ ]:
plt.figure()
#plt.plot(energy, N)
plt.plot(final_energies, N_at_CMS,'.',label='# muons at CMS/s')
#plt.xscale('log')
#plt.yscale('log')
plt.xlim(0,25)

plt.legend()


In [ ]:
plt.figure()
#plt.plot(energy, N)
plt.plot(final_energies, N_at_CMS,label='# muons at CMS/s')
#plt.xscale('log')
#plt.yscale('log')

plt.legend()